In [1]:
%load_ext autoreload
from keras.models import Sequential
from keras.layers.recurrent import LSTM
from keras.layers import Dense, Dropout
from sklearn.model_selection import train_test_split
from sklearn import metrics
from datetime import datetime
from src.average_precision import calculate_score
import numpy as np
import pickle
from keras.preprocessing import sequence
%autoreload 2
from src.average_precision import calculate_delta

Using TensorFlow backend.


In [2]:
X_train = pickle.load( open( "data/X_train.pickle", "rb" ) )
y_train = np.array(pickle.load( open( "data/y_train.pickle", "rb" ) ))

In [3]:
X_train =sequence.pad_sequences(X_train, maxlen=5, dtype='float')

In [6]:
X_delta =calculate_delta(X_train, y_train)

In [8]:
X_delta = np.array(X_delta)

In [9]:
X_delta.shape

(938418, 24)

In [4]:
X_train, X_test, y_train, y_test = train_test_split(
                                    X_train, y_train, test_size=0.20, random_state=42)

In [10]:
X_train.shape

(938418, 5, 46)

In [11]:
model = Sequential()
model.add(LSTM(256, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
model.add(Dropout(0.2))
model.add(LSTM(128, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y_train.shape[1], activation="softmax"))
model.compile(loss="categorical_crossentropy", optimizer="adam")

In [12]:
model.fit(X_train, y_train, nb_epoch=10, batch_size=512)

Epoch 1/20
938418/938418 [==============================] - 47s - loss: 1.2908    
Epoch 2/20
938418/938418 [==============================] - 48s - loss: 1.2107    
Epoch 3/20
938418/938418 [==============================] - 47s - loss: 1.2009    
Epoch 4/20
938418/938418 [==============================] - 48s - loss: 1.1970    
Epoch 5/20
938418/938418 [==============================] - 47s - loss: 1.1944    
Epoch 6/20
938418/938418 [==============================] - 45s - loss: 1.1929    
Epoch 7/20
938418/938418 [==============================] - 45s - loss: 1.1917    
Epoch 8/20
938418/938418 [==============================] - 46s - loss: 1.1906    
Epoch 9/20
938418/938418 [==============================] - 47s - loss: 1.1897    
Epoch 10/20
938418/938418 [==============================] - 48s - loss: 1.1891    
Epoch 11/20
938418/938418 [==============================] - 48s - loss: 1.1883    
Epoch 12/20
938418/938418 [==============================] - 47s - loss: 1.1876    
E

In [14]:
preds = np.round(model.predict_proba(X_test), 2)

187680/187684 [============================>.] - ETA: 0s

In [15]:
score = calculate_score(y_test, preds, X_test)
print(score)
print(metrics.classification_report(y_test, np.round(preds)))

0.722341715572
             precision    recall  f1-score   support

          0       1.00      0.18      0.31        22
          1       1.00      0.50      0.67         2
          2       0.62      0.70      0.66    112198
          3       1.00      0.10      0.19        68
          4       0.97      0.08      0.15     14604
          5       0.98      0.99      0.99      1519
          6       0.97      0.30      0.45      1627
          7       0.99      0.24      0.39     20459
          8       0.99      0.36      0.52      6701
          9       0.80      0.06      0.11        67
         10       0.83      0.03      0.05       194
         11       0.96      0.12      0.22      6126
         12       0.97      0.27      0.42     15339
         13       0.98      0.04      0.08      2962
         14       0.98      0.07      0.13       910
         15       1.00      0.15      0.27      1474
         16       1.00      0.72      0.83       426
         17       1.00      0.

/home/valentin/bin/anaconda3/envs/tensorflow/lib/python3.5/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [81]:
fp = "models/lstm_{0}_SCORE_{1}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S"), np.round(score,5)) 
model.save(fp)

In [13]:
fp = "models/lstm_{0}".format(datetime.now().strftime("%Y-%m-%d %H:%M:%S")) 
model.save(fp)